# Venture Funding with Deep Learning

You work as a risk management associate at Alphabet Soup, a venture capital firm. Alphabet Soup’s business team receives many funding applications from startups every day. This team has asked you to help them create a model that predicts whether applicants will be successful if funded by Alphabet Soup.

The business team has given you a CSV containing more than 34,000 organizations that have received funding from Alphabet Soup over the years. With your knowledge of machine learning and neural networks, you decide to use the features in the provided dataset to create a binary classifier model that will predict whether an applicant will become a successful business. The CSV file contains a variety of information about these businesses, including whether or not they ultimately became successful.


In [49]:
# Imports
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder

import warnings
warnings.filterwarnings('ignore')

%config completer.use_jedi = False

---

## Prepare the data to be used on a neural network model

In [2]:
#Mounting our google drive on google colab to access our data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
filepath = '/content/drive/MyDrive/venture_funding_with_deep_learning/Resources/applicants_data.csv'

applicant_data_df = pd.read_csv(Path(filepath))

# Review the DataFrame
applicant_data_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
# Review the data types associated with the columns
applicant_data_df.dtypes


EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [5]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns = ['EIN','NAME'])

# Review the DataFrame
applicant_data_df.head(20)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [6]:
# Create a list of categorical variables
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes == "object"].index)

# Display the categorical variables list
categorical_variables


['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
# Create a OneHotEncoder instance and encode the categorcal variables using OneHotEncoder
enc = OneHotEncoder(sparse_output = False)
encoded_df = pd.DataFrame(enc.fit_transform(applicant_data_df[categorical_variables]), columns = enc.get_feature_names_out(categorical_variables))
encoded_df.head()


,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat([applicant_data_df[list(applicant_data_df.dtypes[applicant_data_df.dtypes != "object"].index)],
                       encoded_df],
                       axis = 1)

# Review the Dataframe
encoded_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [9]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df['IS_SUCCESSFUL']

# Display a sample of y
y.head()


0    1
1    1
2    0
3    1
4    1
Name: IS_SUCCESSFUL, dtype: int64

In [10]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(['IS_SUCCESSFUL'], axis = 1)

# Review the features DataFrame
X.head()


,STATUS,ASK_AMT,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,APPLICATION_TYPE_T2,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Split the features and target sets into training and testing datasets.


In [11]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)


### Using scikit-learn's `StandardScaler` to scale the features data.

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


---

## Compile and Evaluate a Binary Classification Model Using a Neural Network

In [13]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Define the number of neurons in the output layer
number_output_neurons = 1

# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2


In [14]:
# Review the number hidden nodes in the first, second & output layer
print(f'no. of nodes in input: {hidden_nodes_layer2}, hidden layers :{hidden_nodes_layer1} & {hidden_nodes_layer2} & o/p layer: {number_output_neurons}')


no. of nodes in input: 29, hidden layers :58 & 29 & o/p layer: 1


In [15]:
# Create the Sequential model instance
nn = Sequential()

# Add the hidden layers output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))
nn.add(Dense(units = hidden_nodes_layer2, activation = 'relu'))
nn.add(Dense(units = 1, activation = 'sigmoid'))


In [16]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


###  Compile and fit the model using `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [17]:
# Compile the Sequential model
nn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')


In [18]:
# Fit the model using 50 epochs and the training data
model = nn.fit(X_train_scaled, y_train, epochs = 100)


Epoch 1/100
804/804 [==============================] - 10s 4ms/step - loss: 0.5775 - accuracy: 0.7189
Epoch 2/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5536 - accuracy: 0.7296
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5502 - accuracy: 0.7312
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5488 - accuracy: 0.7322
Epoch 5/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5468 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5449 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5445 - accuracy: 0.7332
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5432 - accuracy: 0.7332
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5425 - accuracy: 0.7362
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5419 - accur

### Evaluate the model using the test data to determine the model’s loss and accuracy.


In [19]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5609 - accuracy: 0.7306 - 582ms/epoch - 2ms/step
Loss: 0.560901939868927, Accuracy: 0.7306122183799744


### Save and export your model to an HDF5 file, and name the file `AlphabetSoup.h5`.


In [20]:
# Set the model's file path
file_path = 'AlphabetSoup.h5'

# Export your model to a HDF5 file
nn.save(file_path)


---

## Optimize the neural network model


### Alternative Model 1

In [21]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [22]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 1

In [23]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

58

In [28]:
#In this model we've increased the nodes in 2nd hidden layer to same as 1st and used LakyRelU instead of Relu
# Create the Sequential model instance
nn_A1 = Sequential()

# Add the hidden layers output layer to the model specifying the number of output neurons and activation function
nn_A1.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'LeakyReLU'))
nn_A1.add(Dense(units = hidden_nodes_layer1, activation = 'LeakyReLU'))
nn_A1.add(Dense(units = 1, activation = 'sigmoid'))

In [29]:
# Compile the Sequential model
nn_A1.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')


In [30]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
804/804 [==============================] - 6s 4ms/step - loss: 0.5814 - accuracy: 0.7156
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5594 - accuracy: 0.7267
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5533 - accuracy: 0.7309
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5522 - accuracy: 0.7294
Epoch 5/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5503 - accuracy: 0.7313
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5497 - accuracy: 0.7319
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5479 - accuracy: 0.7326
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7318
Epoch 9/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5467 - accuracy: 0.7314
Epoch 10/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5460 - accuracy: 0.7326

#### Alternative Model 2

In [31]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [32]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 1

In [33]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A2) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

58

In [34]:
#In this model we've increased the hidden layers
# Create the Sequential model instance
nn_A2 = Sequential()

# Add the hidden layers output layer to the model specifying the number of output neurons and activation function
nn_A2.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))
nn_A2.add(Dense(units = 64, activation = 'relu'))
nn_A2.add(Dense(units = 64, activation = 'relu'))
nn_A2.add(Dense(units = 1, activation = 'sigmoid'))


In [35]:
# Compile the model
nn_A2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')


In [36]:

# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled, y_train, epochs = 100)


Epoch 1/100
804/804 [==============================] - 7s 3ms/step - loss: 0.5728 - accuracy: 0.7218
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7273
Epoch 3/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5488 - accuracy: 0.7305
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5475 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5453 - accuracy: 0.7345
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5444 - accuracy: 0.7336
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5430 - accuracy: 0.7365
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5423 - accuracy: 0.7352
Epoch 9/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5417 - accuracy: 0.7359
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5409 - accura

#### Alternative Model 3

In [44]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [45]:
# Define the number of neurons in the output layer
number_output_neurons_A3 = 1

In [47]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A3 = (number_input_features + number_output_neurons_A3) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A3

58

In [50]:
#In this model we've added the dropout layer and increased the nodes in the 1st hidden layers
# Create the Sequential model instance
nn_A3 = Sequential()

# Add the hidden layers output layer to the model specifying the number of output neurons and activation function
nn_A3.add(Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu'))
nn_A3.add(Dense(units = 128, activation = 'relu'))
nn_A3.add(Dense(units = 64, activation = 'relu'))
nn_A3.add(Dropout(0.2))
nn_A3.add(Dense(units = 1, activation = 'sigmoid'))


In [51]:
# Compile the model
nn_A3.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = 'accuracy')


In [52]:

# Fit the model
fit_model_A3 = nn_A3.fit(X_train_scaled, y_train, epochs = 80)


Epoch 1/80
804/804 [==============================] - 6s 4ms/step - loss: 0.5748 - accuracy: 0.7205
Epoch 2/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5549 - accuracy: 0.7313
Epoch 3/80
804/804 [==============================] - 3s 4ms/step - loss: 0.5511 - accuracy: 0.7317
Epoch 4/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5495 - accuracy: 0.7318
Epoch 5/80
804/804 [==============================] - 4s 5ms/step - loss: 0.5473 - accuracy: 0.7320
Epoch 6/80
804/804 [==============================] - 3s 4ms/step - loss: 0.5465 - accuracy: 0.7347
Epoch 7/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7337
Epoch 8/80
804/804 [==============================] - 3s 3ms/step - loss: 0.5454 - accuracy: 0.7345
Epoch 9/80
804/804 [==============================] - 4s 5ms/step - loss: 0.5440 - accuracy: 0.7356
Epoch 10/80
804/804 [==============================] - 3s 4ms/step - loss: 0.5432 - accuracy: 0.7349

### Step 2: After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [53]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 1s - loss: 0.5609 - accuracy: 0.7306 - 682ms/epoch - 3ms/step
Loss: 0.560901939868927, Accuracy: 0.7306122183799744


In [54]:
print("Alternative Model 1 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy =nn_A1.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 1 Results
268/268 - 0s - loss: 0.5574 - accuracy: 0.7305 - 470ms/epoch - 2ms/step
Loss: 0.557421863079071, Accuracy: 0.7304956316947937


In [55]:
print("Alternative Model 2 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 2 Results
268/268 - 0s - loss: 0.5992 - accuracy: 0.7287 - 480ms/epoch - 2ms/step
Loss: 0.599246084690094, Accuracy: 0.7287463545799255


In [56]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A3.evaluate(X_test_scaled,y_test,verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
268/268 - 1s - loss: 0.5820 - accuracy: 0.7303 - 814ms/epoch - 3ms/step
Loss: 0.5819894075393677, Accuracy: 0.7302623987197876


### Step 3: Save each of your alternative models as an HDF5 file.


In [42]:
# Set the file path for the first alternative model
file_path = 'nn_A1_model.h5'

# Export your model to a HDF5 file
nn_A1.save(file_path)



In [43]:
# Set the file path for the second alternative model
file_path = 'nn_A2_model.h5'

# Export your model to a HDF5 file
nn_A2.save(file_path)


In [ ]:
# Set the file path for the third alternative model
file_path = 'nn_A3_model.h5'

# Export your model to a HDF5 file
nn_A3.save(file_path)